In [ ]:
%matplotlib widget

import warnings
warnings.filterwarnings('ignore')

import os
import glob
import astropy.units as u
from astropy.coordinates import SkyCoord
from sunpy.map import Map
from sunpy.net import Fido, attrs as a
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

plt.rcParams['figure.figsize'] = [7,7]
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['savefig.format'] = 'png'
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['savefig.facecolor'] = 'white'

data_dir = '/home/mnedal/data'
savedir = '/home/mnedal/repos/dias_work'

# os.makedirs(f'{data_dir}/png/aia/131A/runratio', exist_ok=True)
# os.makedirs(f'{data_dir}/png/aia/171A/runratio', exist_ok=True)
# os.makedirs(f'{data_dir}/png/aia/193A/runratio', exist_ok=True)
# os.makedirs(f'{data_dir}/png/aia/211A/runratio', exist_ok=True)

In [ ]:
start_time = '2024-05-14T17:15:00' # '2024-05-14T17:15:00'
end_time   = '2024-05-14T20:00:00' # '2024-05-14T19:00:00'
channel = 195

# 171, 195, 284, and 304 A

In [ ]:
query = (a.Time(start_time, end_time),
         # a.Instrument('SECCHI'),
         a.Instrument('EUVI'),
         a.Source('STEREO_A'),
         a.Wavelength(channel*u.angstrom),
         a.Sample(1*u.min)
         # a.Level(2)
        )

In [ ]:
# stereo_a = Fido.search(a.Time("2007/3/1", "2007/3/2"), a.Instrument("EUVI"), a.Satellite("STEREO_A"), a.Source("BEACON"))

In [ ]:
result = Fido.search(*query)

In [ ]:
result

In [ ]:
stereo_query = Fido.fetch(result, path=f'{savedir}/SECCHI/{channel}A/')

In [ ]:
stereo_map = stereo_query[0]

sta_map = Map(stereo_map)
# sta_map.peek()

fig = plt.figure(figsize=[7,7])
ax = fig.add_subplot(111, projection=sta_map)
sta_map.plot(axes=ax)
ax.grid(False)
fig.tight_layout()
plt.show()

## Import data files

In [ ]:
filenames = sorted(glob.glob(f'{savedir}/SECCHI/{channel}A/*.fts'))
stereo_maps = []
for file in filenames:
    m = Map(file)
    stereo_maps.append(m)

In [ ]:
idx = 7
m = stereo_maps[idx]

top_right   = SkyCoord(-870*u.arcsec, 450*u.arcsec, frame=m.coordinate_frame)
bottom_left = SkyCoord(-1000*u.arcsec, 350*u.arcsec, frame=m.coordinate_frame)

m_submap = m.submap(bottom_left, top_right=top_right)

fig = plt.figure()
ax = fig.add_subplot(projection=m_submap)
img = m_submap.plot(axes=ax,
              # vmin=1e3,
              vmax=20e3,
              # clip_interval=(1, 99.999)*u.percent
             )
# fig.colorbar(img, shrink=0.5, pad=0.02)
ax.grid(False)
fig.tight_layout()
plt.show()

In [ ]:
list_submaps = []
for m in stereo_maps:
    top_right   = SkyCoord(-870*u.arcsec, 550*u.arcsec, frame=m.coordinate_frame)
    bottom_left = SkyCoord(-1270*u.arcsec, 200*u.arcsec, frame=m.coordinate_frame)
    
    # top_right   = SkyCoord(-870*u.arcsec, 450*u.arcsec, frame=m.coordinate_frame)
    # bottom_left = SkyCoord(-1000*u.arcsec, 350*u.arcsec, frame=m.coordinate_frame)
    
    m_submap = m.submap(bottom_left, top_right=top_right)
    list_submaps.append(m_submap)

maps_seq = Map(list_submaps, sequence=True)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection=maps_seq.maps[0])

def update_frame(i):
    """
    Function to update each frame of the animation.
    """
    maps_seq.maps[i].plot(axes=ax,
                          vmax=20e3,
                          # clip_interval=(1, 99.999)*u.percent
                         )
    ax.grid(False)

ani = FuncAnimation(fig, update_frame, frames=len(maps_seq.maps), repeat=False)
fig.tight_layout()
plt.show()

In [ ]:
# Export the frames as PNG images
print(f'Length of the maps list: {len(maps_seq)}\n')

for i, m in enumerate(maps_seq):
    fig = plt.figure()
    ax = fig.add_subplot(projection=m)
    m.plot(axes=ax, vmax=20e3)
    ax.grid(False)
    fig.tight_layout()
    fig.savefig(f"{data_dir}/png/secchi/{channel}A/EUVI{channel}_{m.meta['date-obs']}.png", format='png', dpi=300, bbox_inches='tight')
    # fig.savefig(f"{data_dir}/png/secchi/{channel}A_zoom/EUVI{channel}_{m.meta['date-obs']}.png", format='png', dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Image {i} at {m.meta['date-obs']} is done")

## Another way to download EUVI data

In [ ]:
import pandas as pd

# Define the start and end datetimes
start_datetime = '2024-05-14 17:15:00'
end_datetime   = '2024-05-14 20:00:00'

# Generate the datetime range with hourly intervals
datetime_strings = pd.date_range(start=start_datetime, end=end_datetime, freq='1min').strftime('%Y-%m-%d %H:%M:%S').tolist()

# Print the resulting list
datetime_strings

In [ ]:
import hvpy
from sunpy.time import parse_time


start_time = '2024-05-14T17:20:00' # '2024-05-14T17:15:00'
end_time   = '2024-05-14T20:00:00' # '2024-05-14T19:00:00'
channel = 195

# 'EUVI_A_171',
# 'EUVI_A_195',
# 'EUVI_A_284',
# 'EUVI_A_304',
# 'EUVI_B_171',
# 'EUVI_B_195',
# 'EUVI_B_284',
# 'EUVI_B_304',

In [ ]:
euvi171_file = hvpy.save_file(hvpy.getJP2Image(parse_time(end_time).datetime,
                                               hvpy.DataSource.EUVI_A_171.value),
                              f'{data_dir}/SECCHI/EUVI_171A_{start_time}.jp2', overwrite=True)

euvi171_map = Map(euvi171_file)

fig = plt.figure()
ax = fig.add_subplot(projection=euvi171_map)
euvi171_map.plot(axes=ax)
ax.grid(False)
fig.tight_layout()
plt.show()

In [ ]:
# for t in datetime_strings:
#     print(t)
#     file = hvpy.save_file(hvpy.getJP2Image(parse_time(t).datetime,
#                                            hvpy.DataSource.EUVI_A_171.value),
#                           f'{data_dir}/SECCHI/EUVI_171A_{t}.jp2', overwrite=True)
#     m = Map(file)
#     fig = plt.figure()
#     ax = fig.add_subplot(projection=m)
#     m.plot(axes=ax)
#     ax.grid(False)
#     fig.tight_layout()
#     plt.show()